# OSdaMage preprocessing step
In this step, the OpenStreetMap planet file (europe.osm.pbf ~20 Gb) is cut into small regions according to the European NUTS-3 classification, in a few steps.
1. For each NUTS-3 region in the NUTS-3 shapefile, a .poly file is made
2. For each NUTS-3 region, an extract is made containing all OSM data of that region (with the help of the OSMconvert tool)

In [1]:
import geopandas as gpd
import os as os
import sys
import time as time

from pathos.multiprocessing import Pool 

sys.path.append("..") #import folder which is one level higher

from preproc_functions import *
from utils_functions import load_config
from postproc_functions import NUTS_0_list,NUTS_up,NUTS_3_remote

#this is where the output of the postprocessing is to be stored, i.e. a seperate .osm.pbf per region
#todo: missing the path to the Europe dump
pbf_files_dir = load_config()['paths']['osm_data'] 
input_path = load_config()['paths']['input_data'] #general input data
NUTS3_shape = load_config()['filenames']['NUTS3-shape'] # The shapefile with the NUTS-3 regions in Europe
planet_path = load_config()['paths']['osm_planetpath'] #The path the the Europe planet dump
poly_files_dir = load_config()['paths']['osm_polyfiles'] #Where the results (outcome region chuncks) of this step are to be saved
osm_convert_tool = os.path.join(input_path,"osmconvert64.exe") #Path to the executable

# 1. Construct the .poly files from the NUTS-3 shapefile

In [3]:
#Remove remote overseas territories
filter_out =  NUTS_3_remote(Overseas=True,Creta=True,Spain=True)

### Make the poly files for these regions

In [5]:
poly_files_europe(poly_files_dir,os.path.join(input_path,NUTS3_shape),filter_out)

# 2. Iterate over the .poly files and create OSM chunks per region

In [3]:
#RUN FOR MULTIPLE FILES

#assuming that all NUTS-3 regions DO have a corresponding poly file
regions = [(x.split('.')[0]) for x in os.listdir(poly_files_dir)] #list of region names derived from the .poly files in the folder
list_of_dirs = []

for region in regions:
    area_poly = os.path.join(poly_files_dir,"{}.poly".format(region)) #path to the input poly file
    area_pbf = os.path.join(pbf_files_dir,"{}.osm.pbf".format(region)) #path to the output .osm.pbf file
    #Next line is necessary to enable multiprocessing because the Pool function can handle only 1 argument
    list_of_dirs.append([osm_convert_tool,planet_path,area_poly,area_pbf]) #Each list item contain another list which for the arguments for the clip function

In [ ]:
#EXPLANATION OF THE SYNTAX
#OSM.clip_osm(osm_convert_path,planet_path,area_poly,area_pbf) #this would be the simple way to call the function, not using a list
#OSM.clip_osm_multi(dirs) #this would call the multi_function using one list containing all the same arguments

start = time.time()

with Pool(6) as pool: 
    pool.map(clip_osm_multi,list_of_dirs[0:10],chunksize=1) 
end = time.time()

print("The script runned for: {} seconds".format(end-start))

<b> Known issue: this step can be very slow, especially for complex geometries. Below some suggestions for speeding it up can be found. This step will be improved in the GMTRA model, keep an eye on: https://github.com/ElcoK/gmtra </b>

# Suggestions for error handling:



### Optionally, do some extra preprocessing for problematic regions (for example with much very small islands in front of the coast)

Below some code snippets to change too complicated shapefiles (code not recently checked); can be done with GIS as well. 

In [ ]:
#RUN FOR ONE FILE
region = "NO053"
region_poly = os.path.join(poly_files_dir,"{}.poly".format(region))
region_pbf = os.path.join(pbf_files_dir,"{}.osm.pbf".format(region))
clip_osm(osm_convert_tool,planet_path,region_poly,region_pbf)

In [ ]:
#Simplify

region = "NO071"

region_poly = os.path.join(poly_files_dir,"{}.poly".format(region))
region_poly_simple =  os.path.join(poly_files_dir,"{}_simple.poly".format(region))

polygon = 
remove_tiny_shapes(region_poly)

In [ ]:
problems_shp = os.path.join(input_path,"N3_problems.shp")
p = gpd.read_file(problems_shp)
q = p.copy()
q.at[0,'geometry'] = remove_tiny_shapes2(q.iloc[0])
q.at[1,'geometry'] = remove_tiny_shapes2(q.iloc[1])
q.to_file(os.path.join(input_path,"N3_problems_simplified.shp"))

In [9]:
def remove_tiny_shapes2(x):
    """This function will remove the small shapes of multipolygons. Will reduce the size of the file.
    
    Arguments:
        x {feature} -- a geometry feature (Polygon) to simplify. Countries which are very 
        large will see larger (unhabitated) islands being removed.
    
    Keyword Arguments:
        regionalized {bool} -- set to True to allow for different threshold settings (default: {False})
    """
    threshold = 5000**2 #Threshold is 5000*5000 m

    # save remaining polygons as new multipolygon for the specific country
    new_geom = []
    for y in x.geometry:
        if y.area > threshold:
            new_geom.append(y)

    return MultiPolygon(new_geom)